In [1]:
import os
import shutil
import tensorflow as tf

from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks, optimizers
import numpy as np
from google.colab import drive
import tensorflow_datasets as tfds #imported to get the plant_village dataset
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers import GlobalAvgPool2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

drive.mount('/content/drive/')





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
base_folder = "/content/drive/My Drive/DL_Dataset3"
train_dir = os.path.join(base_folder, "train")
test_dir = os.path.join(base_folder, "test")
validate_dir = os.path.join(base_folder, "validate")




def img_data(dir_path, target_size, batch_size, class_lst, preprocess_func=None):
    gen_object = ImageDataGenerator(preprocessing_function=preprocess_func) if preprocess_func else ImageDataGenerator()
    return gen_object.flow_from_directory(
        dir_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='sparse',
        classes=class_lst,
        shuffle=True
    )

class_list = os.listdir(train_dir)  

train_data_gen = img_data(os.path.join(base_folder, "train"), (224, 224), 500, class_list, tf.keras.applications.efficientnet.preprocess_input)
test_data_gen = img_data(os.path.join(base_folder, "test"), (224, 224), 500, class_list, tf.keras.applications.efficientnet.preprocess_input)
valid_data_gen = img_data(os.path.join(base_folder, "validate"), (224, 224), 500, class_list, tf.keras.applications.efficientnet.preprocess_input)

# Load and set up EfficientNetB0 model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling=None
)
base_model.trainable = False  

# Build and compile the model
model = tf.keras.Sequential([
    base_model,
    GlobalAvgPool2D(),
    BatchNormalization(),
    Dropout(0.5),  
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),  
    Dense(len(class_list), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks and train the model
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_ckpt = ModelCheckpoint("EffNetModel2.keras", save_best_only=True, monitor='val_loss', mode='min')

model.fit(
    train_data_gen,
    validation_data=valid_data_gen,
    epochs=10,
    callbacks=[early_stop, save_ckpt]
)

print('model saved')

Found 22800 images belonging to 38 classes.
Found 9500 images belonging to 38 classes.
Found 5700 images belonging to 38 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 4483s 90s/step - accuracy: 0.1092 - loss: 3.8733 - val_accuracy: 0.5549 - val_loss: 2.6915
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2704s 57s/step - accuracy: 0.4711 - loss: 1.9430 - val_accuracy: 0.7156 - val_loss: 2.0193
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2693s 58s/step - accuracy: 0.6316 - loss: 1.3075 - val_accuracy: 0.7847 - val_loss: 1.5056
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2689s 57s/step - accuracy: 0.7190 - loss: 1.0033 - val_accuracy: 0.8291 - val_loss: 1.1131
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2689s 58s/step - accuracy: 0.7657 - loss: 0.8150 - val_accuracy: 0.8570 - val_loss: 0.8301
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2659s 58s/step - accuracy: 0.7979 - loss: 0.6915 - val_accuracy: 0.8798 - val_loss: 0.6295
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2689s 58s/step - accuracy: 0.8176 - loss: 0.6162 - val_accuracy: 0.8956 - val_loss: 0.4955
Epoch 8/10
37/46 ━━━━━━━━━━━━━━━━━━━━ 6:53 46s/step - accuracy: 0.8349 - loss: 0.5684